<a href="https://colab.research.google.com/github/123shwetarohokale/563-ShwetaR/blob/main/projectoneresubmission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt

def pbc(d, L):
    return np.mod(d + L / 2, L) - L / 2

def distance_matrix(positions, L):
    N = len(positions)
    dist_matrix = np.zeros((N, N))
    for i in range(N):
        for j in range(i+1, N):
            dist_matrix[i, j] = np.linalg.norm(pbc(positions[j] - positions[i], L))
            dist_matrix[j, i] = dist_matrix[i, j]
    return dist_matrix

# Update Lennard-Jones parameters (epsilon=1, sigma=1)
def lj(r, epsilon=1, sigma=1):
    return 4 * epsilon * ((sigma / r) ** 12 - (sigma / r) ** 6)

def autocorr1D(array):
    ft = np.fft.rfft(array - np.average(array))
    acorr = np.fft.irfft(ft * np.conjugate(ft)) / (len(array) * np.var(array))
    dt = np.where(acorr < 0)[0][0]
    nsamples = len(array) // dt
    acorr = acorr[dt:nsamples]
    return nsamples, dt

class MethaneSimulation:
    def __init__(self, N=100, L=50, T=300, maxD=0.1, maxSteps=1000, debug=False, seed=2):
        np.random.seed(seed)
        self.N = N
        self.L = L
        self.T = T
        self.maxD = maxD
        self.accept = 0
        self.reject = 0
        self.rcut = 2.5  # Adjusted for Lennard-Jones spheres (use sigma=1)
        self.pos = np.random.uniform(low=-L/2, high=L/2, size=(N, 3))  # Initial positions
        self.debug = debug
        self.maxSteps = maxSteps
        self.energies = np.zeros(maxSteps)

        self.dist_matrix = distance_matrix(self.pos, L)  # Precompute distance matrix
        self.E = self.calculate_energy()

        for i in range(self.maxSteps):
            self.trial()
            if self.debug:
                print("{:03}:   {:.2f}".format(i, self.E))
            self.energies[i] = self.E

    def calculate_energy(self):
        E = 0
        for i in range(self.N - 1):
            for j in range(i + 1, self.N):
                r = self.dist_matrix[i, j]
                if r <= self.rcut:
                    E += lj(r)
        return E

    def trial(self):
        i = np.random.randint(self.N)
        oldP = np.copy(self.pos[i])
        oldE = self.E
        oldDistMatrix = self.dist_matrix.copy()

        d = np.random.uniform(low=-self.maxD, high=self.maxD, size=3)
        newP = pbc(oldP + d, self.L)
        self.pos[i] = newP

        self.dist_matrix = distance_matrix(self.pos, self.L)  # Update the distance matrix

        newE = self.calculate_energy()
        dE = newE - oldE

        if np.random.uniform() <= np.exp(-dE / self.T):
            if self.debug:
                print("accept")
            self.accept += 1
            self.E = newE
        else:
            if self.debug:
                print("reject")
            self.pos[i] = oldP
            self.dist_matrix = oldDistMatrix  # Revert the distance matrix
            self.reject += 1
            self.E = oldE

# Methane-specific values (using density and adjusting N accordingly)
densities = [0.5, 0.7]
temperatures = [0.5, 2.0]
samples = 20  # Number of independent samples

for density in densities:
    for T in temperatures:
        N = int(100 * density)  # Adjust N based on density
        V = N / density
        L = V ** (1/3)

        print(f"Running simulation for density = {density}, T = {T}, N = {N}, L = {L}")

        energies = []
        for _ in range(samples):
            # Initialize the simulation for each sample
            s = MethaneSimulation(N=N, L=L, T=T, maxSteps=10000, debug=False, seed=23)
            energies.append(s.energies.mean() / N)  # Store average energy per particle

        avg_energy = np.mean(energies)
        std_energy = np.std(energies)
        print(f"Average energy per particle: {avg_energy:.4f} ± {std_energy:.4f}")


Running simulation for density = 0.5, T = 0.5, N = 50, L = 4.641588833612778


<ipython-input-1-a8505e893608>:76: RuntimeWarning: overflow encountered in exp
  if np.random.uniform() <= np.exp(-dE / self.T):


Average energy per particle: 15466.8364 ± 0.0000
Running simulation for density = 0.5, T = 2.0, N = 50, L = 4.641588833612778
Average energy per particle: 15467.5062 ± 0.0000
Running simulation for density = 0.7, T = 0.5, N = 70, L = 4.641588833612778


KeyboardInterrupt: 